Importing modules

In [ ]:
import pandas as pd
import zipfile



Trying to clean the first 100 lines

In [ ]:
b_inds = []
g_inds = []
b_cols = ['TRIP COMMENTS',
          'SPECIES COMMENTS',
          'COUNTRY CODE',
          'STATE CODE',
          'COUNTY CODE',
          'LOCALITY',
          'LOCALITY ID',
          'LOCALITY TYPE',
          'BREEDING CODE',
          'BREEDING CATEGORY',
          'BEHAVIOR CODE',
          'AGE/SEX',
          'ATLAS BLOCK',
          'GLOBAL UNIQUE IDENTIFIER',
          'LAST EDITED DATE']
date_ind = -1
lat_ind = -1
long_ind = -1



def is_in(sth, iterable):
    for item in iterable:
        if sth == item:
            return True
    return False

## Calculate n where (year, month, and day) is in the n-th week of the year

def day_calc(year, month, day):
    if (year < 1) or (month < 1) or (day < 1):
        print('Error: wrong date')
        return None
    leap_year = False
    if year%4 == 0:
        leap_year = True
    if month == 1:
        return day
    else:
        if is_in(month, [4,6,9,11]):
            return day_calc(year, month-1, day+30)
        elif month == 2:
            return day_calc(year, month-1, day+28+leap_year)
        else:
            return day_calc(year, month-1, day+31)






with open("Data/amegfi/2019_ebd_amegfi_Feb-2024.txt", 'r',encoding = "utf8") as file:
## We read the first line here
    file.seek(0)
    line = file.readline()
    s_line = line.split(sep = '\t')
    n_cols = len(s_line)
    for ind in range(n_cols):
        for col in b_cols:
            if s_line[ind] == col:
                b_inds.append(ind)
        if s_line[ind] == 'LATITUDE':
            lat_ind = ind
        if s_line[ind] == 'LONGITUDE':
            long_ind = ind
        if s_line[ind] == 'OBSERVATION DATE':
            date_ind = ind
    if (lat_ind == -1) or (long_ind == -1) or (date_ind == -1) or (b_inds == []):
        print('Error: did not read the first line in the beginning')
## We make our new columns here
    g_inds = [ind for ind in range(n_cols) if is_in(ind, b_inds) == False]
    new_line = ''
    for ind in g_inds:
        if (ind == g_inds[0]):
            new_line += s_line[ind]
        elif (ind == g_inds[-1]):
            if s_line[ind] != '\n':
                print('Error: mistake in figuring out columns')
            new_line += ('\t' + 'YEAR\tWEEK\tDAY\n')
        else:
            new_line += ( '\t' + s_line[ind])
    print(new_line)
## We start reading the data here. We drop the bad indices and only add entries with certain latitudes and longitudes
## For reference, Calhoun County, Texas : (28.366165,-96.568331)
## Kittson County, Minnesota : (48.615191, -96.986411)
## Aroostook County, Maine : (46.965179, -68.713711)
## Jacksonvill, Florida : (30,307830, -81.626867)
    for i in range(100):
        line = file.readline()
        s_line = line.split(sep = '\t')
        s_date = s_line[date_ind].split(sep = '-')
        if (28 <= float(s_line[lat_ind]) <= 49) and (-97 <= float(s_line[long_ind]) <= -68):
            new_line = ''
            for ind in g_inds:
                if (ind == g_inds[0]):
                    new_line += s_line[ind]
                elif (ind == g_inds[-1]):
                    day = day_calc(int(s_date[0]),int(s_date[1]),int(s_date[2]))
                    new_line += ('\t' + s_date[0] + '\t' + str(((day - day%7)//7) + 1) + '\t' + str(day) + '\n')
                else:
                    new_line += ( '\t' + s_line[ind])
            year = int(s_date[0])
            if year == 2016:
                print(new_line)
            if year == 2017:
                print(new_line)
            if year == 2018:
                print(new_line)
            if year == 2019:
                print(new_line)
            if year == 2020:
                print(new_line)
            if year == 2021:
                print(new_line)
            if year == 2022:
                print(new_line)
            if year == 2023:
                print(new_line)


Error: mistake in figuring out columns
SCIENTIFIC NAME	SUBSPECIES SCIENTIFIC NAME	EXOTIC CODE	OBSERVATION COUNT	COUNTRY	STATE	COUNTY	IBA CODE	BCR CODE	USFWS CODE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	YEAR	WEEK	YEAR	WEEK	DAY

Spinus tristis			5	Canada	New Brunswick	Madawaska		14		47.3557539	-68.2252640	2019-03-07	15:00:00	obsr948626	S53689111	Traveling	P22	EBIRD	90	5.000		2	1		0	1	0		2019	10	2019	10	66

Spinus tristis			1	Canada	New Brunswick	Madawaska		14		47.4432608	-68.3969783	2019-03-14	16:05:00	obsr1218521	S53847773	Stationary	P21	EBIRD	10			1	1		0	1	0		2019	11	2019	11	73

Spinus tristis			2	Canada	New Brunswick	Madawaska		14		47.2881153	-68.4192609	2019-05-04	14:00:00	obsr1353203	S56003069	Stationary	P21	EBIRD	360			1	1		0	1	0		2019	18	2019	

Now let's do it for the rest of the data.
We delete columns in b_cols, and for a given species, we create text files containing observations that took place within the
latitude range [28,49] and longitude range [-97,68] and a specified year.

In [ ]:
b_inds = []
b_cols = ['TRIP COMMENTS',
          'SPECIES COMMENTS',
          'COUNTRY CODE',
          'STATE CODE',
          'COUNTY CODE',
          'LOCALITY',
          'LOCALITY TYPE',
          'BREEDING CODE',
          'BREEDING CATEGORY',
          'BEHAVIOR CODE',
          'AGE/SEX',
          'ATLAS BLOCK',
          'COMMON NAME',
          'SUBSPECIES COMMON NAME',
          'TAXONOMIC ORDER',
          'CATEGORY',
          'TAXON CONCEPT ID',
          'LAST EDITED DATE']
date_ind = -1
lat_ind = -1
long_ind = -1
proceed = True



def is_in(sth, iterable):
    for item in iterable:
        if sth == item:
            return True
    return False

## Calculate n where (year, month, and day) is in the n-th week of the year

def day_calc(year, month, day):
    if (year < 1) or (month < 1) or (day < 1):
        print('Error: wrong date')
        return None
    leap_year = False
    if year%4 == 0:
        leap_year = True
    if month == 1:
        return day
    else:
        if is_in(month, [5,7,10,12]):
            return day_calc(year, month-1, day+30)
        elif month == 3:
            return day_calc(year, month-1, day+28+leap_year)
        else:
            return day_calc(year, month-1, day+31)




with zipfile.ZipFile("Data/amegfi/ebd_amegfi_smp_relFeb-2024.zip") as myzip:
    with myzip.open('ebd_amegfi_smp_relFeb-2024.txt', 'r') as file:
        baby2017 = open("Data/amegfi/2017_ebd_amegfi_Feb-2024.txt", "w+")
        baby2018 = open("Data/amegfi/2018_ebd_amegfi_Feb-2024.txt", "w+")
        baby2019 = open("Data/amegfi/2019_ebd_amegfi_Feb-2024.txt", "w+")
        baby2020 = open("Data/amegfi/2020_ebd_amegfi_Feb-2024.txt", "w+")
        baby2021 = open("Data/amegfi/2021_ebd_amegfi_Feb-2024.txt", "w+")
        baby2022 = open("Data/amegfi/2022_ebd_amegfi_Feb-2024.txt", "w+")
        baby2023 = open("Data/amegfi/2023_ebd_amegfi_Feb-2024.txt", "w+")
## We read the first line here
        file.seek(0)
        line = file.readline().decode('utf-8')
        s_line = line.split(sep = '\t')
        n_cols = len(s_line)
        for ind in range(n_cols):
            for col in b_cols:
                if s_line[ind] == col:
                    b_inds.append(ind)
            if s_line[ind] == 'LATITUDE':
                lat_ind = ind
            if s_line[ind] == 'LONGITUDE':
                long_ind = ind
            if s_line[ind] == 'OBSERVATION DATE':
                date_ind = ind
        if (lat_ind == -1) or (long_ind == -1) or (date_ind == -1) or (b_inds == []):
            print('Error: did not read the first line in the beginning')
            proceed = False
## We make our new columns here
        new_line = ''
        g_inds = [ind for ind in range(n_cols) if is_in(ind, b_inds) == False]
        first_ind = g_inds[0]
        last_ind = g_inds[-1]
        for ind in g_inds:
            if (ind == first_ind):
                new_line += s_line[ind]
            elif (ind == last_ind):
                if s_line[ind] != '\n':
                    print('Error: mistake in figuring out columns')
                    proceed = False
                new_line += ('\t' + 'YEAR\tWEEK\tDAY\n')
            else:
                new_line += ( '\t' + s_line[ind])
        baby2017.write(new_line)
        baby2018.write(new_line)
        baby2019.write(new_line)
        baby2020.write(new_line)
        baby2021.write(new_line)
        baby2022.write(new_line)
        baby2023.write(new_line)
## We start reading the data here. We drop the bad indices and only add entries with certain latitudes and longitudes
## For reference, Calhoun County, Texas : (28.366165,-96.568331)
## Kittson County, Minnesota : (48.615191, -96.986411)
## Aroostook County, Maine : (46.965179, -68.713711)
## Jacksonvill, Florida : (30,307830, -81.626867)
        i = 0
        while ((i < 1000000000) and proceed):
            line = file.readline().decode('utf-8')
            if line == '':
                print('End of file!')
                break
            s_line = line.split(sep = '\t')
            s_date = s_line[date_ind].split(sep = '-')
            if (28 <= float(s_line[lat_ind]) <= 49) and (-97 <= float(s_line[long_ind]) <= -68):
                new_line = ''
                for ind in g_inds:
                    if (ind == first_ind):
                        new_line += s_line[ind]
                    elif (ind == last_ind):
                        day = day_calc(int(s_date[0]),int(s_date[1]),int(s_date[2]))
                        new_line += ('\t' + s_date[0] + '\t' + str(((day - day%7)//7) + 1) + '\t' + str(day) + '\n')
                    else:
                        new_line += ( '\t' + s_line[ind])
                year = int(s_date[0])
                if year == 2017:
                    baby2017.write(new_line)
                elif year == 2018:
                    baby2018.write(new_line)
                elif year == 2019:
                    baby2019.write(new_line)
                elif year == 2020:
                    baby2020.write(new_line)
                elif year == 2021:
                    baby2021.write(new_line)
                elif year == 2022:
                    baby2022.write(new_line)
                elif year == 2023:
                    baby2023.write(new_line)
            i += 1
        baby2017.close()
        baby2018.close()
        baby2019.close()
        baby2020.close()
        baby2021.close()
        baby2022.close()
        baby2023.close()
        print(i)

End of file!
17550807


In [ ]:
table = pd.read_table("Data/tuftit/2017_ebd_tuftit_Feb-2024.txt", encoding = 'cp1252')


table.info()



C:\Users\corpt\AppData\Local\Temp\ipykernel_29096\2118087396.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_table("Data/tuftit/2017_ebd_tuftit_Feb-2024.txt", encoding = 'cp1252')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628946 entries, 0 to 628945
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   GLOBAL UNIQUE IDENTIFIER    628946 non-null  object 
 1   SCIENTIFIC NAME             628946 non-null  object 
 2   SUBSPECIES SCIENTIFIC NAME  0 non-null       float64
 3   EXOTIC CODE                 0 non-null       float64
 4   OBSERVATION COUNT           628946 non-null  object 
 5   COUNTRY                     628946 non-null  object 
 6   STATE                       628946 non-null  object 
 7   COUNTY                      628940 non-null  object 
 8   IBA CODE                    107378 non-null  object 
 9   BCR CODE                    617724 non-null  float64
 10  USFWS CODE                  15457 non-null   object 
 11  LOCALITY ID                 628946 non-null  object 
 12  LATITUDE                    628946 non-null  float64
 13  LONGITUDE     

In [ ]:
print(table.head())

                         GLOBAL UNIQUE IDENTIFIER     SCIENTIFIC NAME  \
0  URN:CornellLabOfOrnithology:EBIRD:OBS455252249  Baeolophus bicolor   
1  URN:CornellLabOfOrnithology:EBIRD:OBS454552498  Baeolophus bicolor   
2  URN:CornellLabOfOrnithology:EBIRD:OBS479009093  Baeolophus bicolor   
3  URN:CornellLabOfOrnithology:EBIRD:OBS485251776  Baeolophus bicolor   
4  URN:CornellLabOfOrnithology:EBIRD:OBS480833750  Baeolophus bicolor   

   SUBSPECIES SCIENTIFIC NAME  EXOTIC CODE OBSERVATION COUNT COUNTRY    STATE  \
0                         NaN          NaN                 1  Canada  Ontario   
1                         NaN          NaN                 1  Canada  Ontario   
2                         NaN          NaN                 1  Canada  Ontario   
3                         NaN          NaN                 1  Canada  Ontario   
4                         NaN          NaN                 1  Canada  Ontario   

  COUNTY IBA CODE  BCR CODE  ... NUMBER OBSERVERS ALL SPECIES REPORTED  \
